test cases:

1. get the expected output
2. can throw out errors (unfitted, format error, etc.)
3. produce correct (or nearly correct) amount of NAs

In [1]:
import pandas as pd
import numpy as np

In [2]:
%cd /Users/alex/PETsARD

/Users/alex/PETsARD


In [3]:
import PETsARD

In [4]:
loader = PETsARD.Loader(filepath = '[NHANES] B.csv',
                                header_exist = False,
                                header_names = ['gen','age','race','edu','mar','bmi','dep','pir','gh','mets','qm','dia']
                               )
print(loader.data.head(1))

    gen   age   race       edu      mar        bmi  dep  pir   gh  mets  qm  \
0  Male  62.0  White  Graduate  Married  27.799999    0    0  0.0   0.0  Q2   

   dia  
0    1  


In [5]:
df = loader.data

In [6]:
m = PETsARD.Metadata()
m.build_metadata(df)

In [7]:
metadata = {'metadata_col': {
                'gen': {'type': 'categorical', 'na_percentage': 0.0},
                'age': {'type': 'numerical', 'na_percentage': 0.0},
                'race': {'type': 'categorical', 'na_percentage': 0.0},
                'edu': {'type': 'categorical', 'na_percentage': 0.0},
                'mar': {'type': 'categorical', 'na_percentage': 0.0},
                'bmi': {'type': 'numerical', 'na_percentage': 0.0},
                'dep': {'type': 'numerical', 'na_percentage': 0.0},
                'pir': {'type': 'numerical', 'na_percentage': 0.0},
                'gh': {'type': 'numerical', 'na_percentage': 0.0},
                'mets': {'type': 'numerical', 'na_percentage': 0.0},
                'qm': {'type': 'categorical', 'na_percentage': 0.0},
                'dia': {'type': 'numerical', 'na_percentage': 0.0}
                },
             'metadata_global':{
                 'row_num': 4190,
                 'col_num': 12,
                 'na_percentage': 0.0
                }
            }

In [8]:
pm = PETsARD.Processor.HyperProcessor(m)

In [9]:
from PETsARD.Processor.Missingist import *
from PETsARD.Processor.Outlierist import *
from PETsARD.Processor.Encoder import *
from PETsARD.Processor.Scaler import *

In [10]:
# df.loc[[1,2,3,4,5], 'gen'] = np.nan
# df.loc[[10,20,30,40,5], 'age'] = np.nan

In [11]:
# test global outlierist transformation

config_1 = {'missingist': {'gen': Missingist_Drop(), 'age': Missingist_Simple(5)},
            'outlierist': {'gen': None, 'age': Outlierist_ZScore()},
            'encoder': {'gen': Encoder_Label()},
            'scaler': {'age': Scaler_Log()}}

In [12]:
pm.update_config(config_1)

In [13]:
pm.get_config()

{'missingist': {'gen': <PETsARD.Processor.Missingist.Missingist_Drop at 0x153fd99c0>,
  'age': <PETsARD.Processor.Missingist.Missingist_Simple at 0x153fd8a60>,
  'race': <PETsARD.Processor.Missingist.Missingist_Drop at 0x153f8f100>,
  'edu': <PETsARD.Processor.Missingist.Missingist_Drop at 0x153f8ec20>,
  'mar': <PETsARD.Processor.Missingist.Missingist_Drop at 0x153fd9ba0>,
  'bmi': <PETsARD.Processor.Missingist.Missingist_Mean at 0x153fd9c60>,
  'dep': <PETsARD.Processor.Missingist.Missingist_Mean at 0x153fd9e40>,
  'pir': <PETsARD.Processor.Missingist.Missingist_Mean at 0x153fd9f00>,
  'gh': <PETsARD.Processor.Missingist.Missingist_Mean at 0x153fdb640>,
  'mets': <PETsARD.Processor.Missingist.Missingist_Mean at 0x153fdb520>,
  'qm': <PETsARD.Processor.Missingist.Missingist_Drop at 0x153fdb400>,
  'dia': <PETsARD.Processor.Missingist.Missingist_Mean at 0x153fdb280>},
 'outlierist': {'gen': None,
  'age': <PETsARD.Processor.Outlierist.Outlierist_ZScore at 0x153fd87f0>,
  'race': None,


In [14]:
pm.fit(df)

In [15]:
df_transformed = pm.transform(df)

In [16]:
df_inverse = pm.inverse_transform(df_transformed)

In [17]:
df_inverse

,gen,age,race,edu,mar,bmi,dep,pir,gh,mets,qm,dia
0,Female,72.000000,Mexican,11th,Separated,28.6,0.0,2.775558e-17,0.0,0.0,Q1,2.775558e-17
1,Male,46.000000,White,Graduate,Parther,27.6,0.0,2.775558e-17,0.0,0.0,Q3,2.775558e-17
2,Male,45.000004,Other,11th,Never,24.1,0.0,2.775558e-17,0.0,0.0,Q3,2.775558e-17
3,Female,30.000002,Hispanic,College,Parther,26.6,0.0,2.775558e-17,0.0,0.0,Q4,2.775558e-17
4,Female,27.000002,Black,College,Never,38.0,0.0,2.775558e-17,0.0,0.0,Q1,2.775558e-17
...,...,...,...,...,...,...,...,...,...,...,...,...
2144,Female,41.000000,Other,Graduate,Married,20.9,0.0,2.775558e-17,0.0,0.0,Q1,2.775558e-17
2145,Male,34.000000,Black,Graduate,Married,30.9,0.0,2.775558e-17,0.0,0.0,Q3,2.775558e-17
2146,Male,25.000002,Other,Graduate,Never,21.0,0.0,2.775558e-17,0.0,0.0,Q1,2.775558e-17
2147,Female,80.000000,White,College,Widowed,31.0,0.0,2.775558e-17,0.0,0.0,Q1,2.775558e-17


In [18]:
try:
    print(f'Inverse successful:\n{(df_inverse == df).all()}')
    print(f'Numeric close (age): {np.isclose(df_inverse.age, df.age).all()}')
    print(f'Numeric close (bmi): {np.isclose(df_inverse.bmi, df.bmi).all()}')
except:
    print('Not applicable.')

Not applicable.


In [19]:
loader2 = PETsARD.Loader(filepath = '[Adt Income] adult.csv'
                               ,na_values = {k : '?' for k in ['workclass'
                                                              ,'occupation'
                                                              ,'native-country'
                                                              ]
                                            }
                               )
print(loader2.data.head(1))

   age workclass  fnlwgt education  educational-num marital-status  \
0   25   Private  226802      11th                7  Never-married   

          occupation relationship   race gender  capital-gain  capital-loss  \
0  Machine-op-inspct    Own-child  Black   Male             0             0   

   hours-per-week native-country income  
0              40  United-States  <=50K  


In [20]:
df2 = loader2.data

In [21]:
metadata2 = {'metadata_col': {
                'age': {'type': 'numerical', 'na_percentage': 0.0},
                'workclass': {'type': 'categorical', 'na_percentage': 0.057307},
                'fnlwgt': {'type': 'numerical', 'na_percentage': 0.0},
                'education': {'type': 'categorical', 'na_percentage': 0.0},
                'educational-num': {'type': 'numerical', 'na_percentage': 0.0},
                'marital-status': {'type': 'categorical', 'na_percentage': 0.0},
                'occupation': {'type': 'categorical', 'na_percentage': 0.057512},
                'relationship': {'type': 'categorical', 'na_percentage': 0.0},
                'race': {'type': 'categorical', 'na_percentage': 0.0},
                'gender': {'type': 'categorical', 'na_percentage': 0.0},
                'capital-gain': {'type': 'numerical', 'na_percentage': 0.0},
                'capital-loss': {'type': 'numerical', 'na_percentage': 0.0},
                'hours-per-week': {'type': 'numerical', 'na_percentage': 0.0},
                'native-country': {'type': 'categorical', 'na_percentage': 0.017546},
                'income': {'type': 'categorical', 'na_percentage': 0.0}
                },
             'metadata_global':{
                 'row_num': 48842,
                 'col_num': 15,
                 'na_percentage': 0.07411653904426518
                }
            }

In [22]:
m2 = PETsARD.Metadata()
m2.build_metadata(df2)

In [23]:
pm2 = PETsARD.Processor.HyperProcessor(m2)

In [24]:
pm2.get_config()

{'missingist': {'age': <PETsARD.Processor.Missingist.Missingist_Mean at 0x155073d60>,
  'workclass': <PETsARD.Processor.Missingist.Missingist_Drop at 0x155072710>,
  'fnlwgt': <PETsARD.Processor.Missingist.Missingist_Mean at 0x1550720b0>,
  'education': <PETsARD.Processor.Missingist.Missingist_Drop at 0x1550730d0>,
  'educational-num': <PETsARD.Processor.Missingist.Missingist_Mean at 0x155073eb0>,
  'marital-status': <PETsARD.Processor.Missingist.Missingist_Drop at 0x155072c20>,
  'occupation': <PETsARD.Processor.Missingist.Missingist_Drop at 0x155073cd0>,
  'relationship': <PETsARD.Processor.Missingist.Missingist_Drop at 0x155072650>,
  'race': <PETsARD.Processor.Missingist.Missingist_Drop at 0x155072f50>,
  'gender': <PETsARD.Processor.Missingist.Missingist_Drop at 0x1550719c0>,
  'capital-gain': <PETsARD.Processor.Missingist.Missingist_Mean at 0x1550729b0>,
  'capital-loss': <PETsARD.Processor.Missingist.Missingist_Mean at 0x155073dc0>,
  'hours-per-week': <PETsARD.Processor.Missing

In [25]:
pm2.fit(df2)

In [26]:
df_transformed2 = pm2.transform(df2)

In [27]:
df_transformed2

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,-0.995129,0.109511,0.351675,0.826944,-1.197259,0.785157,0.802880,0.796648,0.928926,0.104216,-0.144804,-0.217127,-0.034087,0.294232,0.478505
1,-0.046942,0.543284,-0.945524,0.096572,-0.419335,0.152862,0.916341,0.299083,0.765002,0.459310,-0.144804,-0.217127,0.772930,0.271194,0.738870
2,-0.776316,0.837091,1.394723,0.849938,0.747550,0.265283,0.989803,0.348380,0.291814,0.633020,-0.144804,-0.217127,-0.034087,0.881880,0.951169
3,-1.068066,0.422021,1.704525,0.513579,-0.030373,0.522408,0.646372,0.895466,0.678457,0.790110,-0.144804,-0.217127,-0.034087,0.362989,0.488145
4,-0.192816,0.990213,0.215911,0.676124,1.136512,0.048613,0.425838,0.206668,0.102049,0.514065,-0.144804,-0.217127,-0.034087,0.689554,0.257046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26928,1.047121,0.302884,1.251867,0.742733,1.525474,0.447056,0.328350,0.254852,0.149531,0.557032,-0.144804,-0.217127,-0.034087,0.344482,0.910820
26929,-1.213941,0.098801,1.140952,0.367170,-0.030373,0.621875,0.986391,0.496976,0.220167,0.350435,-0.144804,-0.217127,-0.034087,0.777780,0.220075
26930,-0.849254,0.433357,0.640492,0.869715,0.747550,0.163935,0.958787,0.937427,0.836045,0.854593,-0.144804,-0.217127,-0.195490,0.593693,0.191550
26931,0.098933,0.027210,-0.334178,0.218720,-0.419335,0.096010,0.762986,0.007155,0.113936,0.405086,-0.144804,-0.217127,-0.034087,0.058037,0.807393


In [28]:
df_inverse2 = pm2.inverse_transform(df_transformed2)

In [29]:
df_inverse2

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25.0,NaN,226802.0,11th,7.0,Never-married,NaN,Own-child,Black,Male,0.0,0.0,40.0,United-States,<=50K
1,38.0,Private,89814.0,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States,<=50K
2,28.0,Local-gov,336951.0,Assoc-acdm,12.0,Married-civ-spouse,Protective-serv,Husband,White,Male,0.0,0.0,40.0,United-States,>50K
3,24.0,NaN,369667.0,Some-college,10.0,Never-married,NaN,Unmarried,White,Female,0.0,0.0,40.0,NaN,<=50K
4,36.0,Federal-gov,212465.0,Bachelors,13.0,Married-civ-spouse,Adm-clerical,Husband,White,Male,0.0,0.0,40.0,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26928,53.0,Private,321865.0,Masters,14.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,40.0,United-States,>50K
26929,22.0,Private,310152.0,Some-college,10.0,Never-married,Protective-serv,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K
26930,27.0,Private,257302.0,Assoc-acdm,12.0,Married-civ-spouse,Tech-support,Wife,White,Female,0.0,0.0,38.0,United-States,<=50K
26931,40.0,Private,154374.0,HS-grad,9.0,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0.0,0.0,40.0,United-States,>50K


In [30]:
df2

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,NaN,103497,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
48838,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
48839,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
48840,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [31]:
df_inverse2.isna().mean(axis=0)

age                0.000000
workclass          0.057290
fnlwgt             0.000000
education          0.000000
educational-num    0.000000
marital-status     0.000000
occupation         0.057476
relationship       0.000000
race               0.000000
gender             0.000000
capital-gain       0.000000
capital-loss       0.000000
hours-per-week     0.000000
native-country     0.017525
income             0.000000
dtype: float64

In [32]:
df_inverse2.isna().any(axis=1).mean()

0.07106523595589054

In [33]:
pm.get_changes()

,processor,col,current,default
0,missingist,age,Missingist_Simple,Missingist_Mean
1,outlierist,age,Outlierist_ZScore,Outlierist_IQR
2,encoder,gen,Encoder_Label,Encoder_Uniform
3,scaler,age,Scaler_Log,Scaler_Standard


In [34]:
from sdv.metadata import SingleTableMetadata

In [35]:
sdvmd = SingleTableMetadata()

In [36]:
sdvmd.detect_from_dataframe(df)

In [37]:
t = sdvmd.to_dict()

In [38]:
sdvmd2 = SingleTableMetadata()

In [39]:
mm = m.to_sdv()